In [1]:
# Load all packages
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [26]:
# Create dataframe
df = pd.read_csv("../../Data/20190403/Datalab_Rit_Herkomst_Bestemming_Uur_20190403.csv", sep=";")

# Change column names
df.columns = ['Datum', 'UurgroepOmschrijving', 'VertrekHalteCode',
       'VertrekHalteNaam', 'VertrekLat', 'VertrekLon',
       'AankomstHalteCode', 'AankomstHalteNaam', 'AankomstLat',
       'AankomstLon', 'AantalRitten']
df = df.dropna()

In [29]:
df2 = df[df['VertrekHalteNaam'] == "Noord"]
df2 = df2[df2['AankomstHalteNaam'] == "Station Zuid"]

In [30]:
df2

,Datum,UurgroepOmschrijving,VertrekHalteCode,VertrekHalteNaam,VertrekLat,VertrekLon,AankomstHalteCode,AankomstHalteNaam,AankomstLat,AankomstLon,AantalRitten
4354936,7/22/2018 12:00:00 AM,08:00 - 08:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,15
4355498,7/22/2018 12:00:00 AM,09:00 - 09:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,26
4356249,7/22/2018 12:00:00 AM,10:00 - 10:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,39
4357190,7/22/2018 12:00:00 AM,11:00 - 11:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,53
4358217,7/22/2018 12:00:00 AM,12:00 - 12:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,71
4359337,7/22/2018 12:00:00 AM,13:00 - 13:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,72
4360453,7/22/2018 12:00:00 AM,14:00 - 14:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,86
4361602,7/22/2018 12:00:00 AM,15:00 - 15:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,55
4362715,7/22/2018 12:00:00 AM,16:00 - 16:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,59
4363781,7/22/2018 12:00:00 AM,17:00 - 17:59,ND,Noord,4.931232,52.401172,WTC,Station Zuid,4.875099,52.33908,50
